# Notebook to generate indicators module

In [1]:
from mintalib import core

from inspect import Signature, Parameter

from __project__ import PKGDIR

In [2]:
import importlib.util


def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [3]:
def core_functions():
    return tuple(k for k, v in vars(core).items() if k.isupper() and callable(v))


core_functions()

('AVGPRICE',
 'TYPPRICE',
 'WCLPRICE',
 'MIDPRICE',
 'PRICE',
 'CROSSOVER',
 'CROSSUNDER',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'INVERT_FLAG',
 'UPDOWN_FLAG',
 'LOG',
 'EXP',
 'ROC',
 'DIFF',
 'MIN',
 'MAX',
 'SUM',
 'MAD',
 'STDEV',
 'SMA',
 'EMA',
 'RMA',
 'WMA',
 'DEMA',
 'TEMA',
 'MA',
 'RSI',
 'PLUSDI',
 'MINUSDI',
 'ADX',
 'TRANGE',
 'ATR',
 'NATR',
 'LATR',
 'SAR',
 'CCI',
 'CMF',
 'MFI',
 'BOP',
 'BBANDS',
 'KELTNER',
 'EFFICIENCY_RATIO',
 'KAMA',
 'MACD',
 'PPO',
 'SLOPE',
 'RVALUE',
 'FORECAST',
 'STOCH',
 'STREAK_UP',
 'STREAK_DOWN',
 'EVAL')

In [4]:
# Sample code for indicator

from mintalib.model import Indicator
from mintalib.core import wrap_function


@wrap_function(core.EMA)
class EMA(Indicator):
    same_scale = True

    def __init__(self, period: int, *, item: str = None):
        self.period = period
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period)

In [5]:
PREAMBLE = """# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from . import model

nan = float('nan')

"""


def make_indicator(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = func.__name__.upper()

    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    indent = " " * 4
    indent2 = " " * 8

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    def annotate(param):
        if param.name in ["period"]:
            return param.replace(annotation=int)

        if param.annotation == "str":
            return param.replace(annotation=str)

        if type(p.default) in (int, float, bool):
            return param.replace(annotation=type(p.default))

        return param

    for i, p in enumerate(parameters):
        if i == 0:
            p = Parameter("self", Parameter.POSITIONAL_OR_KEYWORD)
        else:
            p = annotate(p)
        newparams.append(p)

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    args = [argument(p.name) for p in newparams]
    args = ", ".join(args)

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@core.wrap_function({qname})\n"
    code += f"class {name}(model.Indicator):\n"

    if len(newparams) > 1:
        code += indent + f"def __init__{newsig}:\n"
        for p in newparams[1:]:
            name = p.name
            code += indent2 + f"self.{name} = {name}\n"
        code += "\n"

    code += indent + f"def __call__(self, {ftype}):\n"
    code += indent2 + f"return {qname}({args})\n"

    return code


print(make_indicator(core.EMA))

@core.wrap_function(core.EMA)
class EMA(model.Indicator):
    def __init__(self, period: int, *, adjust: bool = False, item: str = None):
        self.period = period
        self.adjust = adjust
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period, adjust=self.adjust, item=self.item)



In [6]:
def test_indicator(func, name=None, verbose=False):
    code = make_indicator(func, name=name, preamble=True, verbose=verbose)
    print(code)
    return new_module("mintalib.functions", code)


test_indicator(core.EMA)

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from . import model

nan = float('nan')

@core.wrap_function(core.EMA)
class EMA(model.Indicator):
    def __init__(self, period: int, *, adjust: bool = False, item: str = None):
        self.period = period
        self.adjust = adjust
        self.item = item

    def __call__(self, series):
        return core.EMA(series, period=self.period, adjust=self.adjust, item=self.item)



<module 'mintalib.functions'>

In [7]:
def make_indicators(functions=core_functions()):
    buffer = PREAMBLE

    for name in functions:
        func = getattr(core, name)
        text = make_indicator(func)
        buffer += text + "\n\n"

    buffer += "__all__ = [name for name in dir() if name.isupper()]\n"

    return buffer


code = make_indicators()
print(code[:256], "...")

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from . import model

nan = float('nan')

@core.wrap_function(core.AVGPRICE)
class AVGPRICE(model.Indicator):
    def __call__(self, prices):
        return core.AVGPRICE(pr ...


In [8]:
code = make_indicators()

module = new_module("mintalib.indicators", code)

dir(module)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'DEMA',
 'DIFF',
 'EFFICIENCY_RATIO',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FORECAST',
 'INVERT_FLAG',
 'KAMA',
 'KELTNER',
 'LATR',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MFI',
 'MIDPRICE',
 'MIN',
 'MINUSDI',
 'NATR',
 'PLUSDI',
 'PPO',
 'PRICE',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK_DOWN',
 'STREAK_UP',
 'SUM',
 'TEMA',
 'TRANGE',
 'TYPPRICE',
 'UPDOWN_FLAG',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'model',
 'nan']

In [9]:
outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

Updating indicators.py ...


14706